In [9]:
import os
import cv2 as cv
import numpy as np

VIDEO_PATH = "test-videos/mona-lisa-blur.avi"
OUT_PATH   = "results/1.mp4"
os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)

# LK (межкадровый трекинг)
MAX_CORNERS   = 1000
MIN_TRACK_PTS = 120
FB_THR        = 1.5
MIN_INLIERS_LK = 20

lk_params = dict(
    winSize=(21, 21),
    maxLevel=5,
    criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 30, 0.01)
)

# Re-detect (по 1-му кадру)
RATIO         = 0.80
MIN_MATCHES   = 35
MIN_INLIERS_RD = 18
RANSAC_THR    = 5.0
# BLUR_SIGMAS   = [0.0, 1.0, 2.0, 3.0]


In [ ]:
clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
det = cv.AKAZE_create()
matcher = cv.BFMatcher(cv.NORM_HAMMING)

def prep(gray):
    return clahe.apply(gray)

def reseed_klt(gray, corners):
    mask = np.zeros(gray.shape[:2], np.uint8)
    cv.fillPoly(mask, [corners.reshape(-1,2).astype(np.int32)], 255)
    return cv.goodFeaturesToTrack(gray, MAX_CORNERS, 0.01, 7, blockSize=7, mask=mask)


def redetect(gray, des0, kp0):
    g = prep(gray)
    kp, des = det.detectAndCompute(g, None)
    if des is None or des0 is None:
        return None, 0, 0

    pairs = matcher.knnMatch(des0, des, k=2)

    good = []
    for m, n in pairs:
        if m.distance < RATIO * n.distance:
            good.append(m)

    if len(good) < MIN_MATCHES:
        return None, len(good), 0

    src = np.float32([kp0[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)  
    dst = np.float32([kp[m.trainIdx].pt  for m in good]).reshape(-1, 1, 2)  

    Hm, inl = cv.findHomography(src, dst, cv.RANSAC, RANSAC_THR)
    inl_cnt = int(inl.sum()) if inl is not None else 0

    if Hm is None or inl_cnt < MIN_INLIERS_RD:
        return None, len(good), inl_cnt

    return Hm, len(good), inl_cnt



In [ ]:
cap = cv.VideoCapture(VIDEO_PATH)
ok, frame0 = cap.read()
if not ok:
    raise RuntimeError("Не читается первый кадр")

H, W = frame0.shape[:2]
fps = cap.get(cv.CAP_PROP_FPS) or 25.0
writer = cv.VideoWriter(OUT_PATH, cv.VideoWriter_fourcc(*"mp4v"), fps, (W, H))

gray0 = cv.cvtColor(frame0, cv.COLOR_BGR2GRAY)
g0 = prep(gray0)
kp0, des0 = det.detectAndCompute(g0, None)


corners0 = np.float32([[0,0],[W-1,0],[W-1,H-1],[0,H-1]]).reshape(-1,1,2)
corners  = corners0.copy()

prev_gray = gray0
p_prev = reseed_klt(prev_gray, corners)
lost = False


In [ ]:
while True:
    ok, frame = cap.read()
    if not ok:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # 1) Трекинг LK между соседними кадрами (пока не lost)
    if not lost:
        if p_prev is None or len(p_prev) < MIN_TRACK_PTS:
            p_prev = reseed_klt(prev_gray, corners)

        # если точек всё равно нет — значит LK не из чего считать -> сразу lost
        if p_prev is None or len(p_prev) < 8:
            lost = True
        else:
            p_prev = p_prev.astype(np.float32)
            p_next, st, _  = cv.calcOpticalFlowPyrLK(prev_gray, gray, p_prev, None, **lk_params)
            p_back, st2, _ = cv.calcOpticalFlowPyrLK(gray, prev_gray, p_next, None, **lk_params)

            fb = np.linalg.norm(p_prev - p_back, axis=2).reshape(-1)
            good = (st.reshape(-1) == 1) & (st2.reshape(-1) == 1) & (fb < FB_THR)

            p0g = p_prev[good]
            p1g = p_next[good]

            if len(p0g) >= 8:
                Hm, inl = cv.findHomography(p0g, p1g, cv.RANSAC, 3.0)
                if Hm is not None and inl is not None and int(inl.sum()) >= MIN_INLIERS_LK:
                    inl = inl.reshape(-1).astype(bool)
                    corners = cv.perspectiveTransform(corners, Hm)
                    p_prev = p1g[inl].reshape(-1,1,2)
                    prev_gray = gray
                else:
                    lost = True
            else:
                lost = True

    if lost:
        Habs, n_good, n_inl = redetect(gray, des0, kp0)
        if Habs is not None:
            corners = cv.perspectiveTransform(corners0, Habs)
            p_prev = reseed_klt(gray, corners)
            prev_gray = gray
            lost = False

    vis = frame.copy()
    color = (0, 255, 0) if not lost else (0, 0, 255)
    cv.polylines(vis, [corners.reshape(-1,2).astype(np.int32)], True, color, 3)
    writer.write(vis)

cap.release()
writer.release()
print("Saved:", OUT_PATH)


Saved: results/1.mp4
